In [1]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import plotly.express as px



In [2]:
from app.flipside.models import Flipside
from app.oracles.models import Oracle
from app.constants import (
    oracle_feeds_lookup, oracle_feeds, 
    phase_windows, timestamp_windows,
    filenames_filtered,
    liquidators
    )
from app.aave.models import AaveBalancePlotter
from app.combo.models import Combo
from app.swaps.models import DexSwaps
from app.visuals.plotly_helper import combined_action_layout, combined_action_layout_time

In [3]:
from app.constants import phase_windows

In [17]:
filenames = {
    'transfers_from': 'crv_transfers_from_avi_partner',
    'transfers_to': 'crv_transfers_to_participants' 
}



In [18]:
fs = Flipside(filenames)

Loading Files...
crv_transfers_from_avi_partner
crv_transfers_to_participants

Formatting Dataframes...
transfers_from BLOCK_NUMBER
transfers_from AMOUNT
transfers_from AMOUNT_USD
transfers_from TOKEN_PRICE
transfers_from DECIMALS
transfers_from RAW_AMOUNT
transfers_to BLOCK_NUMBER
transfers_to AMOUNT
transfers_to AMOUNT_USD
transfers_to TOKEN_PRICE
transfers_to DECIMALS
Integer out of range. at position 40
transfers_to RAW_AMOUNT


In [31]:
df_avi = fs.dataframes['transfers_from']
df_avi.keys()

df_liq = fs.dataframes['transfers_to']

df_liq = df_liq[df_liq.TO_ADDRESS.isin(liquidators)]

df_liq.keys()


Index(['BLOCK_NUMBER', 'BLOCK_TIMESTAMP', 'TX_HASH',
       'ORIGIN_FUNCTION_SIGNATURE', 'ORIGIN_FROM_ADDRESS', 'ORIGIN_TO_ADDRESS',
       'CONTRACT_ADDRESS', 'FROM_ADDRESS', 'TO_ADDRESS', 'RAW_AMOUNT',
       'DECIMALS', 'SYMBOL', 'TOKEN_PRICE', 'AMOUNT', 'AMOUNT_USD',
       'HAS_DECIMAL', 'HAS_PRICE', '_LOG_ID', 'INGESTED_AT',
       '_INSERTED_TIMESTAMP'],
      dtype='object')

In [38]:

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.update_layout(
    title=f"Transfers by Avi",
#     xaxis_title="X Axis Title",
#     yaxis_title="Y Axis Title",
#     legend_title="Legend Title",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    )
)
fig = fig.add_trace(
    go.Scatter(
        x=df_avi.BLOCK_TIMESTAMP,
        y=df_avi.AMOUNT,
        name="Transfers Avi",
        mode='markers',
        marker_color="blue",
    ),
    # secondary_y=True
)
fig.show()


## Setup

In [20]:
df_filtered = df_avi.loc[
            (df_avi["BLOCK_NUMBER"] > 15962578) &
            (df_avi["BLOCK_NUMBER"] < 16023400-420)
        ]

out_avi = df_filtered.groupby(["TO_ADDRESS", "SYMBOL"]).AMOUNT.sum().reset_index()
out_avi = out_avi.sort_values("AMOUNT", axis = 0, ascending = False)
out_avi

,TO_ADDRESS,SYMBOL,AMOUNT
0,0x6cc5f688a315f3dc28a7781717a9a798a59fda7b,CRV,1.155013e+07


## Attack

In [22]:
df_filtered = df_avi.loc[
            (df_avi["BLOCK_NUMBER"] > 16023400-420) &
            (df_avi["BLOCK_NUMBER"] < 16025280)
        ]

out_avi = df_filtered.groupby(["TO_ADDRESS", "SYMBOL"]).AMOUNT.sum().reset_index()
out_avi = out_avi.sort_values("AMOUNT", axis = 0, ascending = False)
out_avi

,TO_ADDRESS,SYMBOL,AMOUNT
0,0x6cc5f688a315f3dc28a7781717a9a798a59fda7b,CRV,60000000.0


In [10]:
title = "test"
fig = px.bar(out_avi,
        x=out_avi.TO_ADDRESS,
        y=out_avi.AMOUNT,
        # color='FROM_ADDRESS',
        title=f"Sum Liquidated Collateral By Liquidators (Tokens): {title}",
        # facet_col='BORROWER_ADDRESS',
        # facet_col_wrap=3
            )
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
# fig.update_layout(autotypenumbers='convert types')
fig.show()

In [39]:
out_liq = df_liq.groupby(["FROM_ADDRESS"]).AMOUNT.sum().reset_index()
out_liq = out_liq.sort_values("AMOUNT", axis = 0, ascending = False)
out_liq

,FROM_ADDRESS,AMOUNT
14,0x8301ae4fc9c624d1d396cbdaa1ed877821d7c511,4.066855e+07
15,0x8dae6cb04688c62d939ed9b68d32bc62e49970b1,2.851645e+07
13,0x777777c9898d384f785ee44acfe945efdff5f3e0,1.558081e+07
16,0x9008d19f58aabd9ed0d60971565aa8510560ab41,7.632786e+06
4,0x28c6c06298d514db089934071355e5743bf21d60,6.422597e+06
7,0x4c83a7f819a5c37d64b4c5a2f8238ea082fa1f4e,6.253842e+06
9,0x57e04786e231af3343562c062e0d058f25dace9e,3.703193e+06
21,0xa81011ae274ef6debd3bdab634102c7b6c2c452d,3.560000e+06
26,0xdbf5e9c5206d0db70a90108bf936da60221dc080,3.485176e+06
2,0x21a31ee1afc51d94c2efccaa2092ad1028285549,1.927286e+06


In [41]:
title = "test"
fig = px.bar(out_liq,
        x=out_liq.FROM_ADDRESS,
        y=out_liq.AMOUNT,
        # color='TO_ADDRESS',
        title=f"Sum CRV Transfers To Liquidators (Tokens): {title}",
        # facet_col='TO_ADDRESS',
        # facet_col_wrap=4
            )
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
# fig.update_layout(autotypenumbers='convert types')
# fig.update_layout(autotypenumbers='convert types')
fig.show()


In [ ]:
len(out_liq['TO_ADDRESS'].unique())

In [25]:
df_liq = df_liq[df_liq.FROM_ADDRESS.isin(['0x8dae6cb04688c62d939ed9b68d32bc62e49970b1'])]


In [26]:
out_liq = df_liq.groupby(["TO_ADDRESS"]).AMOUNT.sum().reset_index()
out_liq = out_liq.sort_values("AMOUNT", axis = 0, ascending = False)
out_liq

,TO_ADDRESS,AMOUNT
5,0xc37704a457b1ee87eb657cae584a34961e86acac,1.370124e+07
2,0x8bc110db7029197c3621bea8092ab1996d5dd7be,1.100000e+07
4,0x9bae78d1c67826cde91b20b49690589ed0879fc7,1.700000e+06
0,0x058b10cbe1872ad139b00326686ee8ccef274c58,5.700010e+05
6,0xd542aa8f1789edf123ad816c1b59ed9fed15c50e,5.000010e+05
3,0x95ecfcc073f1d768be35839dd27724a0aed78e60,5.000000e+05
1,0x4f381fb46dfde2bc9dcae2d881705749b1ed6e1a,3.000000e+05
7,0xd911560979b78821d7b045c79e36e9cbfc2f6c6f,1.802139e+05
8,0xdfd3bd446f1b7fd96dc995126ee845af0b1254cd,6.500000e+04


In [27]:
title = "test"
fig = px.bar(out_liq,
        x=out_liq.TO_ADDRESS,
        y=out_liq.AMOUNT,
        # color='TO_ADDRESS',
        title=f"Sum CRV Withdrawn By Liquidators (Tokens): {title}",
        # facet_col='TO_ADDRESS',
        # facet_col_wrap=4
            )
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
# fig.update_layout(autotypenumbers='convert types')
# fig.update_layout(autotypenumbers='convert types')
fig.show()
